In [ ]:
## imports

import numpy as np
import pandas as pd

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)
pd.set_option('display.max_columns', 100)
from lightgbm import LGBMClassifier
from datetime import datetime

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.model_selection import GridSearchCV,StratifiedKFold,KFold
from sklearn.preprocessing import LabelEncoder
seed = 45
#seed =145
% matplotlib inline
plt.style.use('fivethirtyeight')

In [ ]:
### read data
PATH='F:/AV/WNS'
train_csv = 'train_LZdllcl.csv'
test_csv = 'test_2umaH9m.csv'
submit_csv = 'sample_submission_M0L0uXE.csv'

### read train, test and submission files
train = pd.read_csv(f'{PATH}/{train_csv}')
test = pd.read_csv(f'{PATH}/{test_csv}')
submission = pd.read_csv(f'{PATH}/{submit_csv}')

print("Shape of {}:{} {}:{} {}:{}".format('train',train.shape,'test',test.shape,'submission',submission.shape))

In [ ]:
### inspect data

train.head()

In [ ]:
### class balance

train['is_promoted'].value_counts()

## so approx 10% of past employees have been promoted

In [ ]:
### lets check if there is any repeat in employees

len(train['employee_id'].unique()) == train.shape[0]

### so all IDs are unique

In [ ]:
## null values

null_columns=train.columns[train.isnull().any()]
train[null_columns].isnull().sum()

## so 2 columns have null values

In [ ]:
### inspect null value columns
a = train[(train.education.isnull())]
_ = train[(train.education.isnull() | train.previous_year_rating.isnull())]

print(a.shape,_.shape)

In [ ]:
set(a.index).intersection(set(_.index)) == set(a.index)

### so everywhere where education is not present prev year rating is also not present, but vice-versa is not true

In [ ]:
_.head(10)

In [ ]:
### check avg % of people promoted with NA in previous ye rating vs without NA

print(np.mean(_['is_promoted']),np.mean(train[~train.education.isnull()]['is_promoted']))

In [ ]:
### check avg % of people promoted with NA in education vs without NA

print(np.mean(a['is_promoted']),np.mean(train[~train.education.isnull()]['is_promoted']))

In [ ]:
## For education we will use unknown for all missing values and 9999 for prev year training

train['education'] = train.education.fillna('unknown')
train['previous_year_rating'] = train.previous_year_rating.fillna(9999)

test['education'] = test.education.fillna('unknown')
test['previous_year_rating'] = test.previous_year_rating.fillna(9999)

In [ ]:
### merge dataframes for ease of processing
Y = train['is_promoted'].values
train.drop('is_promoted',inplace=True,axis=1)
train['train'] = 'train'
test['train'] = 'test'
merged = pd.concat([train,test])
merged.shape

In [ ]:
cat_cols = [i for i in merged.columns if merged[i].dtypes == 'object']+['KPIs_met >80%','awards_won?']
cat_cols

In [ ]:
cat_cols.remove('train')
print(cat_cols)

In [ ]:
import pandas as pd
import numpy as np
import gc

import mlcrate as mlc
import pickle as pkl

from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dot, Reshape, Add, Subtract
from keras import objectives
from keras import backend as K
from keras import regularizers 
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2

In [ ]:
import tensorflow as tf
import random as rn
def init_seeds(seed):
    os.environ['PYTHONHASHSEED'] = '0'

    # The below is necessary for starting Numpy generated random numbers
    # in a well-defined initial state.

    np.random.seed(seed)

    # The below is necessary for starting core Python generated random numbers
    # in a well-defined state.

    rn.seed(seed)

    # Force TensorFlow to use single thread.
    # Multiple threads are a potential source of
    # non-reproducible results.
    # For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

    from keras import backend as K

    # The below tf.set_random_seed() will make random number generation
    # in the TensorFlow backend have a well-defined initial state.
    # For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

    tf.set_random_seed(seed)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)
    return sess

In [ ]:
merged.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:
features = ['department', 'region', 'education']
f_size  = [len(merged[f].unique()) + 1 for f in features]
f_size

In [ ]:
merged_1 = merged.copy()

In [ ]:
merged = MultiColumnLabelEncoder(columns = features).fit_transform(merged)

In [ ]:
merged.head()

In [ ]:
X = merged.groupby(features)['employee_id'].count()
X.head()

In [ ]:
X = X.unstack().fillna(0)
X.head()

In [ ]:
X = X.stack().astype('float32')
X.head()

In [ ]:
X = np.log1p(X).reset_index()
X.head()

In [ ]:
X.columns=features + ['num']
X.head()

In [ ]:
# X = MultiColumnLabelEncoder(columns = features).fit_transform(X)
# print(merged.head())

In [ ]:
X_train = [X[f].values for f in features]
y_train = (X[['num']].values).astype('float32')

In [ ]:
X.shape

In [ ]:
(X.num > 0).mean()

In [ ]:
k_latent = 2
embedding_reg = 0.0002
kernel_reg = 0.1

def get_embed(x_input, x_size, k_latent):
    if x_size > 0: #category
        embed = Embedding(x_size, k_latent, input_length=1, 
                          embeddings_regularizer=l2(embedding_reg))(x_input)
        embed = Flatten()(embed)
    else:
        embed = Dense(k_latent, kernel_regularizer=l2(embedding_reg))(x_input)
    return embed

def build_model_1(X, f_size):
    dim_input = len(f_size)
    
    input_x = [Input(shape=(1,)) for i in range(dim_input)] 
     
    biases = [get_embed(x, size, 1) for (x, size) in zip(input_x, f_size)]
    
    factors = [get_embed(x, size, k_latent) for (x, size) in zip(input_x, f_size)]
    
    s = Add()(factors)
    
    diffs = [Subtract()([s, x]) for x in factors]
    
    dots = [Dot(axes=1)([d, x]) for d,x in zip(diffs, factors)]
    
    x = Concatenate()(biases + dots)
    x = BatchNormalization()(x)
    output = Dense(1, activation='relu', kernel_regularizer=l2(kernel_reg))(x)
    model = Model(inputs=input_x, outputs=[output])
    opt = Adam(clipnorm=0.5)
    model.compile(optimizer=opt, loss='mean_squared_error')
    output_f = factors + biases
    model_features = Model(inputs=input_x, outputs=output_f)
    return model, model_features

In [ ]:
model, model_features = build_model_1(X_train, f_size)
model.summary()

In [ ]:
model_features.summary()

In [ ]:
n_epochs = 10000
P = 17
try:
    del sess
except:
    pass
sess = init_seeds(0)

batch_size = 2**P
print(batch_size)
model, model_features = build_model_1(X_train, f_size)
earlystopper = EarlyStopping(patience=250, verbose=1)

model.fit(X_train,  y_train, 
          epochs=n_epochs, batch_size=batch_size, verbose=1, shuffle=True, 
          validation_data=(X_train, y_train), 
          sample_weight=None,
          callbacks=[earlystopper],
         )

In [ ]:
X_pred = model_features.predict(X_train, batch_size=batch_size)

In [ ]:
factors = X_pred[:len(features)]

biases = X_pred[len(features):2*len(features)]

for f, X_p in zip(features, factors):
    for i in range(k_latent):
        X['%s_fm_factor_%d' % (f, i)] = X_p[:,i]

for f, X_p in zip(features, biases):
    X['%s_fm_bias' % (f)] = X_p[:,0]

X.head()

In [ ]:
X.head()

In [ ]:
#X.to_csv('F:/AV/WNS/Interaction_feats.csv',index=False)

In [ ]:
X =pd.read_csv('F:/AV/WNS/Interaction_feats.csv')
X.head()

In [ ]:
_ = X.copy()
_.head()

In [ ]:
import gc
gc.collect()

In [ ]:
# for feats in tqdm(features):
#     feat1 = feats+'_fm_factor_0'
#     feat2 = feats+'_fm_factor_1'
#     bias = feats+'_fm_bias'
#     right_df = X[[feats,feat1,feat2,bias]]
#     right_df = right_df.drop_duplicates()
#     _ = merged[[feats]].merge(right_df,on=feats)
#     _.to_csv('F:/AV/WNS/{}.csv'.format(feats),index=False)
#     del right_df,_
#     gc.collect()

In [ ]:
for feats in tqdm(features):
    feat1 = feats+'_fm_factor_0'
    feat2 = feats+'_fm_factor_1'
    bias = feats+'_fm_bias'
    right_df = X[[feats,feat1,feat2,bias]]
    right_df = right_df.drop_duplicates()
    merged = merged.merge(right_df,on=feats)
    merged[[feats]] = merged_1[[feats]].values
    del right_df
    gc.collect()

In [ ]:
cat_cols

In [ ]:
def pre_process(df,cat_cols,drop_cats=True):
    
    if drop_cats:
        df.drop(cat_cols,inplace=True,axis=1)
        
    try:
        one_hot_encoded_training_predictors = pd.get_dummies(df[cat_cols])
        temp = df.drop(cat_cols,inplace=False,axis=1)
        _ = pd.concat([temp,one_hot_encoded_training_predictors],1)
        new_tr, new_tst = _[_['train']=='train'],_[_['train']=='test']
        new_tr.drop('train',inplace=True,axis=1)
        new_tst.drop('train',inplace=True,axis=1)
        del temp
        return new_tr, new_tst
    except:
        new_tr, new_tst = df[df['train']=='train'],df[df['train']=='test']
        new_tr.drop('train',inplace=True,axis=1)
        new_tst.drop('train',inplace=True,axis=1)
        return new_tr, new_tst
    
    #return new_tr, new_tst

In [ ]:
train_OHE,test_OHE = pre_process(merged,cat_cols,drop_cats=True)

In [ ]:
merged.tail()

In [ ]:
def kfold_lightgbm(train_df,test_df, target,num_folds, stratified = False, debug= False,modelname="lightgbm_0",drop_cat=True):
    # Divide in training/validation and test data
    train_df = train_df
    test_df = test_df
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    gc.collect()

    
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['employee_id','index','train']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], target)):
        train_x, train_y = train_df[feats].iloc[train_idx], target[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], target[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            nthread=4,
            n_estimators=5000,
            learning_rate=0.001,
#            num_leaves=34,
#            colsample_bytree=0.9,
#            subsample=0.8715623,
            max_depth=8,
            reg_alpha=1,
            reg_lambda=1,
#            min_split_gain=0.0222415,
#            min_child_weight=50,
            silent=-1,
            verbose=-1, )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 100, early_stopping_rounds= 200)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
        print(oof_preds[:5])
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d F-score : %.6f' % (n_fold + 1, sklearn.metrics.f1_score(valid_y, (oof_preds[valid_idx]>0.3).astype(int))))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full f1 score %.6f' % sklearn.metrics.f1_score(target, (oof_preds>0.3).astype(int)))
    
    # Write submission file and plot feature importance
    if not debug:
        _ = datetime.now().strftime('%Y%m%d%H%M%S')
        Fname = 'F:/AV/WNS/submission/'+str(modelname)+'_'+str(_)+'.csv'
        submission['is_promoted'] = sub_preds
        submission[['employee_id', 'is_promoted']].to_csv(Fname, index= False)
        oof = pd.DataFrame(oof_preds)
        score = sklearn.metrics.f1_score(target, (oof_preds>0.3).astype(int))
        oof.columns = [modelname+'_'+str(round(score,4))]
        OOF_Fname = 'F:/AV/WNS/oof/'+str(modelname)+'_'+str(_)+'.csv'
        oof.to_csv(OOF_Fname,index=False)
    #display_importances(feature_importance_df)
    return 

In [ ]:
import gc
oof = kfold_lightgbm(train_OHE,test_OHE, Y,num_folds=10, stratified = True, debug= False,modelname="lightgbm_5fld_interactionfeatsonly")